<a href="https://colab.research.google.com/github/Gopichand14052004/BATCH_5_NLP/blob/main/2203a51015_ass_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Example text data
text = """Once upon a time, there was a little girl named Red Riding Hood. She loved to visit her grandmother, who lived in the woods. One day, her mother asked her to take a basket of goodies to her grandmother. On her way through the woods, she met a big bad wolf who wanted to eat her."""

# Tokenization and preparing sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for i in range(1, len(tokenizer.texts_to_sequences([text])[0])):
    n_gram_sequence = tokenizer.texts_to_sequences([text])[0][:i+1]
    input_sequences.append(n_gram_sequence)

# Pad sequences
max_seq_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

# Inputs and labels
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


# Define Transformer-based model using MultiHeadAttention
def transformer_block(vocab_size, seq_len):
    inputs = tf.keras.layers.Input(shape=(seq_len,))
    embedding = tf.keras.layers.Embedding(vocab_size, 64)(inputs)

    # Multi-Head Attention Layer
    attention_output = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=64)(embedding, embedding)
    attention_output = tf.keras.layers.LayerNormalization()(attention_output + embedding)

    # Feed-forward network
    ff_output = tf.keras.layers.Dense(128, activation='relu')(attention_output)
    ff_output = tf.keras.layers.Dense(64)(ff_output)
    ff_output = tf.keras.layers.LayerNormalization()(ff_output + attention_output)

    outputs = tf.keras.layers.GlobalAveragePooling1D()(ff_output)
    outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(outputs)

    model = tf.keras.models.Model(inputs, outputs)
    return model

# Build model
model = transformer_block(total_words, max_seq_len-1)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model (20, 60, 70 epochs)
epochs_list = [20, 60, 70]
history = []
for epoch in epochs_list:
    print(f"Training with {epoch} epochs...")
    h = model.fit(X, y, epochs=epoch, verbose=1)
    history.append(h)

# Text generation using seed text
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=1)
        output_word = tokenizer.index_word[predicted_word_index[0]]
        seed_text += " " + output_word
    return seed_text

seed_text = "Once upon a time"
generated_text = generate_text(seed_text, 50, model, max_seq_len)
print("Generated text:", generated_text)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 55)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 55, 64)         │          2,624 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 55, 64)         │         33,216 │ embedding[0][0],       │
│ (MultiHeadAttention)      │                        │                │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 55, 64)         │              0 │ multi_head_attention[… │
│                           │                        │                │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 55, 64)         │            128 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 55, 128)        │          8,320 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 55, 64)         │          8,256 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 55, 64)         │              0 │ dense_1[0][0],         │
│                           │                        │                │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 55, 64)         │            128 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d  │ (None, 64)             │              0 │ layer_normalization_1… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 41)             │          2,665 │ global_average_poolin… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 55,337 (216.16 KB)

 Trainable params: 55,337 (216.16 KB)

 Non-trainable params: 0 (0.00 B)

Training with 20 epochs...
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.0225 - loss: 3.9356
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0451 - loss: 3.6212 
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0814 - loss: 3.5370 
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.1491 - loss: 3.4300 
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0936 - loss: 3.4108
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1040 - loss: 3.4126
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.1369 - loss: 3.3333
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.1282 - loss: 3.3469 
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.1161 - loss: 3.3097
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1369 - loss: 3.2440
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.1369 - loss: 3.2038
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - ac